In [4]:
import os
import re

import numpy as np
import pandas as pd
import geopandas as gpd
from tqdm import tqdm
tqdm.pandas()

from shapely.geometry import MultiPolygon, Polygon
from shapely.validation import make_valid

In [9]:
from constants import CEN_YEARS
from constants import YEAR_CODES

mpoly_GTA = gpd.read_file('../data/geo/regions/GTA.gpkg').geometry.to_list()[0]

Prune geometries to select for the GTA-only versions

In [6]:
cen_years_81 = [y for y in CEN_YEARS if y >= 1981]

for year in tqdm(cen_years_81):
    gdf_csd_all = gpd.read_file(f'../data/geo/{year}_csd/csd_{year}.geojson')
    gdf_csd_all['gta_overlap'] = gdf_csd_all.apply(lambda row: row.geometry.intersection(mpoly_GTA).area / row.geometry.area, axis=1)
    gdf_csd_gta = gdf_csd_all[gdf_csd_all.gta_overlap >= 0.25]
    gdf_csd_gta.to_file(f'../data/geo/{year}_csd/csd_gta_{year}.gpkg', driver="GPKG")

100%|██████████| 9/9 [02:19<00:00, 15.47s/it]


In [10]:
# For each year, save those CT's which are in the GTA
for year in tqdm(CEN_YEARS):
    gdf_geo_all = gpd.read_file(f"../data/geo/{year}_ct/ct_{year}.zip")
    gdf_geo_all['gta_overlap'] = gdf_geo_all.apply(lambda row: row.geometry.intersection(mpoly_GTA).area / row.geometry.area, axis=1)
    gdf_geo_gta = gdf_geo_all[gdf_geo_all.gta_overlap > 0.5]
    gdf_geo_gta.to_file(f"../data/geo/{year}_ct/ct_gta_{year}.gpkg", driver='GPKG')

100%|██████████| 15/15 [00:42<00:00,  2.82s/it]
